# IMPORTS & SEEDS

In [1]:
!pip install sentence-transformers==2.7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.3 MB/s eta 0:00:00


In [3]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.3 MB/s eta 0:00:00


In [4]:
import json
import pandas as pd
import torch
import random
import numpy as np
import os
import random


from datasets import Dataset, load_dataset, concatenate_datasets
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer
from google.colab import files

os.environ["WANDB_DISABLED"] = "true"

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# Datasets Loading

## SIB (Topic Classification)

In [5]:
from datasets import load_dataset

# Specify the languages you want to use
languages = languages = ["fra_Latn", "eng_Latn", "deu_Latn", "jpn_Jpan", "zho_Hans", "rus_Cyrl", "ltz_Latn"]

# Load language-specific splits
train_datasets = {}
test_datasets = {}

for lang in languages:
    lang_dataset = load_dataset("mteb/sib200", lang)
    train_datasets[lang] = lang_dataset["train"]
    test_datasets[lang] = lang_dataset["test"]

sib_train_dataset = concatenate_datasets([train_datasets[lang] for lang in languages if lang!='ltz_Latn'])
sib_test_dataset = test_datasets['ltz_Latn']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/48.7k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/205 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/205 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/205 [00:00<?, ?it/s]

...

ltz_Latn.parquet:   0%|          | 0.00/52.1k [00:00<?, ?B/s]

ltz_Latn.parquet:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
zero_shot_template_labels = ["An dësem Beispill geet et em ", "D'Thema vun dësem Text ass ", "Dëst Dokument beschäftegt sech mat "] # just add label at the end.

category_to_lux_map = {
    "science/technology": "Technologie.",
    "travel": "Reesen.",
    "politics": "Politik.",
    "health": "Gesondheet.",
    "entertainment": "Ennerhalung.",
    "geography": "Geographie.",
    "sports": "Sport."
}

class_to_templates = {}
for category in list(set(sib_train_dataset['category'])):
  class_to_templates[category] = []
  for template in zero_shot_template_labels:
    class_to_templates[category].append(template + category_to_lux_map[category])
  class_to_templates[category].append(category_to_lux_map[category][:-1]) # one template that is just the label, no full stop
  class_to_templates[category].append("Hei gëtt iwwer " + category_to_lux_map[category][:-1] + " geschwat.") # one template that label is inbetween

In [7]:
print(class_to_templates)

{'politics': ['An dësem Beispill geet et em Politik.', "D'Thema vun dësem Text ass Politik.", 'Dëst Dokument beschäftegt sech mat Politik.', 'Politik', 'Hei gëtt iwwer Politik geschwat.'], 'geography': ['An dësem Beispill geet et em Geographie.', "D'Thema vun dësem Text ass Geographie.", 'Dëst Dokument beschäftegt sech mat Geographie.', 'Geographie', 'Hei gëtt iwwer Geographie geschwat.'], 'entertainment': ['An dësem Beispill geet et em Ennerhalung.', "D'Thema vun dësem Text ass Ennerhalung.", 'Dëst Dokument beschäftegt sech mat Ennerhalung.', 'Ennerhalung', 'Hei gëtt iwwer Ennerhalung geschwat.'], 'sports': ['An dësem Beispill geet et em Sport.', "D'Thema vun dësem Text ass Sport.", 'Dëst Dokument beschäftegt sech mat Sport.', 'Sport', 'Hei gëtt iwwer Sport geschwat.'], 'health': ['An dësem Beispill geet et em Gesondheet.', "D'Thema vun dësem Text ass Gesondheet.", 'Dëst Dokument beschäftegt sech mat Gesondheet.', 'Gesondheet', 'Hei gëtt iwwer Gesondheet geschwat.'], 'travel': ['An dë

## Paralux ("Paraphrase Identification")

In [8]:
# Load the ParaLux test dataset
paralux_test = load_dataset("fredxlpy/ParaLux", split="test")

# Now you can work with the paralux_test dataset
paralux_data = {}
paralux_data['anchor'] = paralux_test['anchor']
paralux_data['positive'] = paralux_test['paraphrase']
paralux_data['negative'] = paralux_test['not_paraphrase']

README.md:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

benchmark.json:   0%|          | 0.00/86.4k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/312 [00:00<?, ? examples/s]

## Historical Bitext Mining Loading

In [11]:
def load_dataset_local(file_path):
    """
    Loads a dataset (list of dictionaries) from a JSONL file.

    Args:
        file_path (str): Path to the JSONL file.

    Returns:
        list: List of dictionaries loaded from the file.
    """
    dataset = []
    with open(file_path, "r") as file:
        for line in file:
            dataset.append(json.loads(line))
    return dataset

In [13]:
bitext_mining_data_fr_to_lb = load_dataset_local("prepared_bitext_mining_format/bitext_mining_task_fr_to_lb.jsonl")
bitext_mining_data_lb_to_fr = load_dataset_local("prepared_bitext_mining_format/bitext_mining_task_lb_to_fr.jsonl")

bitext_mining_data_en_to_lb = load_dataset_local("prepared_bitext_mining_format/bitext_mining_task_en_to_lb.jsonl")
bitext_mining_data_lb_to_en = load_dataset_local("prepared_bitext_mining_format/bitext_mining_task_lb_to_en.jsonl")

bitext_mining_data_de_to_lb = load_dataset_local("prepared_bitext_mining_format/bitext_mining_task_de_to_lb.jsonl")
bitext_mining_data_lb_to_de = load_dataset_local("prepared_bitext_mining_format/bitext_mining_task_lb_to_de.jsonl")

# Evaluation Functions

In [21]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist

def numpy_cosine_similarity(v1, v2):
    """Compute cosine similarity using NumPy."""
    v1 = np.squeeze(v1)
    v2 = np.squeeze(v2)
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def evaluate_sib_topic_classification_with_templates(model, dataset, class_to_templates):
    """
    Evaluate a model's performance in assigning higher cosine similarity to the correct category.

    Args:
        model: A model object with an `.encode()` method for generating embeddings.
        dataset: The dataset containing examples with 'text' and 'category' fields.
        class_to_templates: A dictionary mapping categories to their list of templates.

    Returns:
        template_performance: A list of performance scores (accuracy) for each template.
        average_performance: The average accuracy across all templates.
    """
    # Precompute template embeddings
    precomputed_template_embeddings = {
        category: model.encode(templates) for category, templates in class_to_templates.items()
    }

    # Placeholder for results
    template_performance = []

    # Get categories and number of templates
    categories = list(class_to_templates.keys())
    num_templates = len(next(iter(class_to_templates.values())))

    # Iterate over templates
    for template_idx in range(num_templates):
        correct_count = 0
        total_count = 0

        for example in dataset:  # Iterate over the dataset
            true_category = example["category"]
            text = example["text"]

            # Generate embedding for the current text
            text_embedding = model.encode([text])

            # Retrieve precomputed template embeddings for the current template index
            template_embeddings = np.array([
                precomputed_template_embeddings[category][template_idx]
                for category in categories
            ])

            # Compute cosine similarities
            similarities = cosine_similarity(text_embedding, template_embeddings).flatten()

            # Identify the predicted category
            predicted_idx = np.argmax(similarities)
            predicted_category = categories[predicted_idx]

            # Check if the predicted category is correct
            if predicted_category == true_category:
                correct_count += 1

            total_count += 1

        # Calculate performance for this template
        accuracy = correct_count / total_count
        template_performance.append(accuracy)

    # Calculate average performance across templates
    average_performance = np.mean(template_performance)

    return template_performance, average_performance


def evaluate_paralux_performance(anchor, positive, negative, model):
    """
    Measures the frequency with which the model's embeddings yield a higher cosine similarity
    between anchor and positive than between anchor and negative examples.

    Parameters:
        anchor (list): List of anchor text strings.
        positive (list): List of positive/paraphrase text strings.
        negative (list): List of negative/not paraphrase text strings.
        model: A model with an `encode` method to generate text embeddings.

    Returns:
        float: The frequency of anchor-positive similarity being higher than anchor-negative similarity.
    """
    # Encode the texts
    anchor_embeddings = model.encode(anchor)
    positive_embeddings = model.encode(positive)
    negative_embeddings = model.encode(negative)

    # Compute cosine similarities
    anchor_positive_sim = cosine_similarity(anchor_embeddings, positive_embeddings).diagonal()
    anchor_negative_sim = cosine_similarity(anchor_embeddings, negative_embeddings).diagonal()

    # Calculate the percentage of samples where positive similarity > negative similarity
    higher_similarity_percentage = round(np.mean(anchor_positive_sim > anchor_negative_sim) * 100, 2)

    return higher_similarity_percentage

def evaluate_historical_bitext_mining_task(bitext_data, model, sentence_embeddings = None):
    """
    Evaluates an embedding model on the bitext-mining task.

    Args:
        bitext_data (list): A list of dictionaries containing the bitext-mining task data.
        model (object): A model with a `.encode` method to compute sentence embeddings.

    Returns:
        float: The accuracy of the model on the bitext-mining task.
    """
    # Precompute embeddings for all unique sentences
    if sentence_embeddings is None:
        unique_sentences = set(
            sentence for entry in bitext_data for sentence in [entry["source_sentence"]] + entry["candidates"]
        )
        sentence_embeddings = {sentence: model.encode(sentence) for sentence in unique_sentences}

    correct_count = 0

    for entry in bitext_data:
        source_embedding = sentence_embeddings[entry["source_sentence"]]
        parallel_embedding = sentence_embeddings[entry["candidates"][0]]
        parallel_similarity = numpy_cosine_similarity([source_embedding], [parallel_embedding])

        # Prepare the embeddings for all candidates
        candidate_embeddings = [sentence_embeddings[candidate] for candidate in entry["candidates"][1:]]

        # Use cdist for one-to-many cosine similarity computation
        similarities = 1 - cdist(candidate_embeddings, source_embedding.reshape(1, -1), metric="cosine").flatten()

        # Check if any similarity exceeds max_similarity
        if np.max(similarities) < parallel_similarity:
            correct_count += 1

    accuracy = round(correct_count / len(bitext_data) * 100,2)
    return accuracy, sentence_embeddings

def run_lux_embeddings_evaluations(
    model,
    model_name,
    sib200_dataset,
    sib200_class_to_templates,
    similarity_data,
    bitext_mining_datasets_de_lb,
    bitext_mining_datasets_fr_lb=None,
    bitext_mining_datasets_en_lb=None,
    run_sib_clf=True,
    run_paralux_clf=True,
    run_bitext_mining=True
):
    """
    Combines template-based model evaluation and similarity frequency measurement.

    Parameters:
        model: A model object with an `.encode()` method for generating embeddings.
        model_name (str): Name of the model for display in the results.
        dataset: The dataset containing examples with 'text' and 'category' fields (required for evaluation).
        class_to_templates: A dictionary mapping categories to their list of templates (required for evaluation).
        similarity_data (dict): A dictionary containing 'anchor', 'positive', and 'negative' lists for similarity check.
        run_evaluation (bool): Whether to run the template-based evaluation.
        run_similarity_check (bool): Whether to run the similarity frequency check.

    Returns:
        dict: A dictionary containing results for evaluation and similarity checks.
    """
    results = {}

    if run_sib_clf:
        template_performance, average_performance = evaluate_sib_topic_classification_with_templates(
            model, sib200_dataset, class_to_templates
        )
        results["Zero Shot SIB"] = round(average_performance * 100,2)
        # print(f"Zero Shot SIB (7 classes) Accuracy for {model_name}: {average_performance*100:.2f}%")

    if run_paralux_clf:
        if not (similarity_data and
                "anchor" in similarity_data and
                "positive" in similarity_data and
                "negative" in similarity_data):
            raise ValueError("A dictionary containing 'anchor', 'positive', and 'negative' lists must be provided for similarity check.")

        paralux_performance = evaluate_paralux_performance(
            similarity_data["anchor"],
            similarity_data["positive"],
            similarity_data["negative"],
            model
        )
        results["PARALux Accuracy"] = paralux_performance
        # print(f"PARALux (300 samples) Accuracy for : {model_name}': {paralux_performance:.2f}%")
    if run_bitext_mining:
        de_lb_accuracy, sentence_embeddings = evaluate_historical_bitext_mining_task(bitext_mining_datasets_de_lb[0], model)
        results["Historical Bitext Mining DE -> LB"] = de_lb_accuracy
        # print(f"DE - > LB Historical Bitext Mining Accuracy (2170 Sentences) for {model_name}: " + str(de_lb_accuracy) + "%")
        lb_de_accuracy, sentence_embeddings = evaluate_historical_bitext_mining_task(bitext_mining_datasets_de_lb[1], model, sentence_embeddings)
        results["Historical Bitext Mining LB -> DE"] = lb_de_accuracy
        # print(f"LB - > DE Historical Bitext Mining Accuracy (2170 Sentences) for {model_name}: "  + str(lb_de_accuracy) + "%")

        fr_lb_accuracy, sentence_embeddings = evaluate_historical_bitext_mining_task(bitext_mining_datasets_fr_lb[0], model)
        results["Historical Bitext Mining FR -> LB"] = fr_lb_accuracy
        # print(f"DE - > LB Historical Bitext Mining Accuracy (471 Sentences) for {model_name}: " + str(fr_lb_accuracy) + "%")
        lb_fr_accuracy, sentence_embeddings = evaluate_historical_bitext_mining_task(bitext_mining_datasets_fr_lb[1], model, sentence_embeddings)
        results["Historical Bitext Mining LB -> FR"] = lb_fr_accuracy

        en_lb_accuracy, sentence_embeddings = evaluate_historical_bitext_mining_task(bitext_mining_datasets_en_lb[0], model)
        results["Historical Bitext Mining EN -> LB"] = en_lb_accuracy
        # print(f"DE - > LB Historical Bitext Mining Accuracy (471 Sentences) for {model_name}: " + str(fr_lb_accuracy) + "%")
        lb_en_accuracy, sentence_embeddings = evaluate_historical_bitext_mining_task(bitext_mining_datasets_en_lb[1], model, sentence_embeddings)
        results["Historical Bitext Mining LB -> EN"] = lb_en_accuracy
        # print(f"LB - > DE Historical Bitext Mining Accuracy (471 Sentences) for {model_name}: "  + str(lb_fr_accuracy) + "%")

    return results

# Train and Evaluate Models - Standard Multiple Negatives Ranking Loss (Batch = 8, Epoch = 1)

## Training Utility Functions

In [22]:
import os
import json
import re
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

class CustomEvaluator:
    def __init__(
        self,
        model_name,
        sib_test_dataset,
        class_to_templates,
        similarity_data,
        bitext_mining_datasets_de_lb,
        bitext_mining_datasets_fr_lb=None,
        bitext_mining_datasets_en_lb=None,
        log_file="evaluation_logs.jsonl"
    ):
        self.model_name = model_name
        self.sib_test_dataset = sib_test_dataset
        self.class_to_templates = class_to_templates
        self.similarity_data = similarity_data
        self.bitext_mining_datasets_de_lb = bitext_mining_datasets_de_lb
        self.bitext_mining_datasets_fr_lb = bitext_mining_datasets_fr_lb
        self.bitext_mining_datasets_en_lb = bitext_mining_datasets_en_lb

        # Get the directory of the log file
        log_dir = os.path.dirname(log_file)
        if log_dir:  # Only create directory if it exists
            os.makedirs(log_dir, exist_ok=True)

        self.log_file = log_file

        # Initialize the log file if it doesn't exist
        if not os.path.exists(self.log_file):
            with open(self.log_file, "w") as f:
                pass  # Create an empty file


    def __call__(
        self, model: SentenceTransformer, output_path: str = None, epoch: int = -1, steps: int = -1
    ) -> dict[str, float]:
        # Run evaluations
        results = run_lux_embeddings_evaluations(
            model=model,
            model_name=self.model_name,
            sib200_dataset=self.sib_test_dataset,
            sib200_class_to_templates=self.class_to_templates,
            similarity_data=self.similarity_data,
            bitext_mining_datasets_de_lb=self.bitext_mining_datasets_de_lb,
            bitext_mining_datasets_fr_lb=self.bitext_mining_datasets_fr_lb,
            bitext_mining_datasets_en_lb=self.bitext_mining_datasets_en_lb,
            run_sib_clf=True,
            run_paralux_clf=True,
            run_bitext_mining=True
        )

        # Create a log entry
        log_entry = {
            "epoch": epoch,
            "steps": steps,
            "results": results
        }

        # Append the log entry to the JSONL file
        with open(self.log_file, "a") as f:
            f.write(json.dumps(log_entry) + "\n")  # Write as a single line

        print(f"Logged evaluation at epoch {epoch}, step {steps} to {self.log_file}")
        return results["Historical Bitext Mining DE -> LB"]

def prepare_training_samples(train_samples_tuple, sample_size=5120, random_sampling=False, sorted_descending=False):
    """
    Prepares training samples for finetuning the embedding models.

    Args:
        train_samples_tuple (list): A list of tuples, each containing a pair of sentences and a custom_id.
        sample_size (int, optional): The number of samples to prepare. Defaults to 3000.
        random_sampling (bool, optional): If True, samples are randomly selected; otherwise, the first `sample_size` samples are taken. Defaults to False.
        sorted_descending (bool, optional): If True, samples are sorted in descending order of (joint) character length. Defaults to False.

    Returns:
        list: A list of `InputExample` objects containing the training samples.
    """
    if sorted_descending:
        train_samples_tuple = sorted(train_samples_tuple, key=lambda pair: len(pair[0]) + len(pair[1]), reverse=True)
    if random_sampling:
        train_samples_tuple = random.sample(train_samples_tuple, min(sample_size, len(train_samples_tuple))) # is also shuffled
    else:
        train_samples_tuple = train_samples_tuple[sample_size]

    train_samples = [InputExample(texts=[sample[0], sample[1]], label=1) for sample in train_samples_tuple]
    return train_samples


def clean_text_from_punctuation(text):
        """
        Cleans text by removing random punctuation and extra whitespace.

        Args:
            text (str): The text to clean.

        Returns:
            str: Cleaned text containing only alphanumeric characters and preserving spaces.
        """
        return re.sub(r'[^a-zA-Z0-9\s]+', '', text).strip().lower()

def extract_parallel_sentences(lines, src_col, tgt_col):
    """
    Extracts parallel sentence pairs (lb, de) and preserves the custom_id from the JSONL lines.

    Args:
        lines (list): A list of dictionaries, where each dictionary represents a line from the file.

    Returns:
        list: A list of tuples containing sentence pairs (lb_sentence, de_sentence, custom_id).
    """
    parallel_sentences = []
    for data in lines:
        custom_id = data.get('custom_id', None)
        translations = data.get('translation', {})
        for sentence_pair in translations:
            if isinstance(sentence_pair, dict):
                src_sentence = sentence_pair.get(src_col)
                tgt_sentence = sentence_pair.get(tgt_col)
                if type(src_sentence) == list or type(tgt_sentence) == list: # to handle a special case of ['']
                    continue
                if (src_sentence and tgt_sentence)  and (len(clean_text_from_punctuation(src_sentence)) >= 5 and len(clean_text_from_punctuation(tgt_sentence)) >= 5): # must contain atleast 5 alphnum characters
                    parallel_sentences.append((src_sentence, tgt_sentence, custom_id))
    return parallel_sentences

## German

### Prepare Training Set

In [23]:
# Example usage to load the datasets
train_file_path = "prepared_training_sentences/lb_de_training_set.jsonl" # Or use the HF Version

# Loading datasets
lb_de_training_set_loaded = load_dataset_local(train_file_path)

# Confirm loading was successful
print(f"Loaded training set size (in articles): {len(lb_de_training_set_loaded)}")

# Extract parallel sentences from the training lines
lb_de_training_sentences = extract_parallel_sentences(lb_de_training_set_loaded, src_col="lb", tgt_col="de")
# Prepare the training samples
lb_de_sampled_training_set = prepare_training_samples(lb_de_training_sentences, sample_size=20160, random_sampling=True)

Loaded training set size (in articles): 2105


###  Historical Only

#### LABSE

##### 20160 Samples (evaluate every 512 samples)

In [24]:
labse_random = SentenceTransformer("sentence-transformers/LaBSE")

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_de_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)

train_loss = losses.MultipleNegativesRankingLoss(model=labse_random)


epochs = 1

custom_evaluator = CustomEvaluator(model_name = "LABSE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="labse_evaluation_logs_de.jsonl")


# Evaluate before training starts (step 0)
custom_evaluator(model=labse_random, output_path="./model_output", epoch=0, steps=0)

labse_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to labse_evaluation_logs_de.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2512 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
files.download("labse_evaluation_logs_de.jsonl")

#### LuxEmbedder

##### 20160 Samples (evaluate every 512 samples)

In [ ]:
luxemb_random = SentenceTransformer("fredxlpy/LuxEmbedder")

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_de_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)

total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)


train_loss = losses.MultipleNegativesRankingLoss(model=luxemb_random)

custom_evaluator = CustomEvaluator(model_name = "LuxEmb", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de],bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="luxembed_evaluation_logs_de.jsonl")
# Fine-tune the model

# Evaluate before training starts (step 0)
custom_evaluator(model=luxemb_random, output_path="./model_output", epoch=0, steps=0)

epochs = 1
# Fine-tune the model with evaluation every 500 steps
luxemb_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to luxembed_evaluation_logs_de.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2512 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 64 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 128 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 192 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 256 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 320 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 384 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 448 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 512 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 576 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 640 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 704 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 768 to luxembed_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 832 to luxembed_evaluation_logs_de.jsonl
Logged evalua

In [ ]:
files.download("luxembed_evaluation_logs_de.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### M-GTE

##### 20160 Samples (evaluate every 512 samples)

In [ ]:
mgte_random = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_de_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.001 * total_steps)


custom_evaluator = CustomEvaluator(model_name = "M-GTE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="mgte_evaluation_logs_de.jsonl")

train_loss = losses.MultipleNegativesRankingLoss(model=mgte_random)

# Evaluate before training starts (step 0)
custom_evaluator(model=mgte_random, output_path="./model_output", epoch=0, steps=0)

# Fine-tune the model
epochs = 1
# Fine-tune the model with evaluation every 500 steps
mgte_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/123k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Logged evaluation at epoch 0, step 0 to mgte_evaluation_logs_de.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2512 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 64 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 128 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 192 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 256 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 320 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 384 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 448 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 512 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 576 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 640 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 704 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 768 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 832 to mgte_evaluation_logs_de.jsonl
Logged evaluation at epoch 0, step 896 to mgte_evaluation_logs_de

In [ ]:
files.download('mgte_evaluation_logs_de.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## French

### Prepare Training Set

In [ ]:
# Example usage to load the datasets
train_file_path = "prepared_training_sentences/lb_fr_training_set.jsonl" # Or use the HF Version

# Loading datasets
lb_fr_training_set_loaded = load_dataset_local(train_file_path)

# Confirm loading was successful
print(f"Loaded training set size (in articles): {len(lb_fr_training_set_loaded)}")

cont_df = load_dataset("fredxlpy/LuxAlign", "lb-fr", split="train")
lb_fr_cont_training_sentences = [
    (lb, fr) for lb, fr in zip(cont_df["lb"], cont_df["fr"])
    if lb and fr
]

# Extract parallel sentences from the training lines
lb_fr_training_sentences = extract_parallel_sentences(lb_fr_training_set_loaded, src_col="lb", tgt_col="fr")
# Prepare the training samples
lb_fr_sampled_training_set = prepare_training_samples(lb_fr_training_sentences, sample_size=20160, random_sampling=True)
# Prepare the training samples
lb_fr_cont_training_set = prepare_training_samples(lb_fr_cont_training_sentences, sample_size=20160, random_sampling=True)

lb_fr_mixed_sentences = lb_fr_sampled_training_set + lb_fr_cont_training_set

Loaded training set size (in articles): 2105
Loaded test set size (in articles): 233


README.md:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

lb_fr.json:   0%|          | 0.00/23.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/89405 [00:00<?, ? examples/s]

### Historical Only

#### LABSE

##### 20160 Samples (evaluate every 512 samples)

In [ ]:
labse_random = SentenceTransformer("sentence-transformers/LaBSE")

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_fr_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)

train_loss = losses.MultipleNegativesRankingLoss(model=labse_random)


epochs = 1

custom_evaluator = CustomEvaluator(model_name = "LABSE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="labse_evaluation_logs_fr.jsonl")

# Evaluate before training starts (step 0)
custom_evaluator(model=labse_random, output_path="./model_output", epoch=0, steps=0)

labse_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to labse_evaluation_logs_fr.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2502 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 64 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 128 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 192 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 256 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 320 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 384 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 448 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 512 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 576 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 640 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 704 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 768 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 832 to labse_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 896 to labse_eval

In [ ]:
files.download("labse_evaluation_logs_fr.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### LuxEmbedder

##### 20160 Samples (evaluate every 512 samples)

In [ ]:
luxemb_random = SentenceTransformer("fredxlpy/LuxEmbedder")

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_fr_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)

total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)


train_loss = losses.MultipleNegativesRankingLoss(model=luxemb_random)

custom_evaluator = CustomEvaluator(model_name = "LuxEmb", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en],  log_file="luxembed_evaluation_logs_fr.jsonl")
# Fine-tune the model
epochs = 1
# Evaluate before training starts (step 0)
custom_evaluator(model=luxemb_random, output_path="./model_output", epoch=0, steps=0)
# Fine-tune the model with evaluation every 500 steps
luxemb_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to luxembed_evaluation_logs_fr.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2502 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 64 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 128 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 192 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 256 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 320 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 384 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 448 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 512 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 576 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 640 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 704 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 768 to luxembed_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 832 to luxembed_evaluation_logs_fr.jsonl
Logged evalua

In [ ]:
files.download("luxembed_evaluation_logs_fr.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### M-GTE

##### 20160 Samples (evaluate every 512 samples)

In [ ]:
mgte_random = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_fr_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.001 * total_steps)


custom_evaluator = CustomEvaluator(model_name = "M-GTE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="mgte_evaluation_logs_fr.jsonl")

train_loss = losses.MultipleNegativesRankingLoss(model=mgte_random)

# Fine-tune the model
epochs = 1

# Evaluate before training starts (step 0)
custom_evaluator(model=mgte_random, output_path="./model_output", epoch=0, steps=0)

# Fine-tune the model with evaluation every 500 steps
mgte_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/123k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Logged evaluation at epoch 0, step 0 to mgte_evaluation_logs_fr.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2502 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 64 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 128 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 192 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 256 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 320 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 384 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 448 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 512 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 576 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 640 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 704 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 768 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 832 to mgte_evaluation_logs_fr.jsonl
Logged evaluation at epoch 0, step 896 to mgte_evaluation_logs_fr

In [ ]:
files.download("mgte_evaluation_logs_fr.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Historical And Contemporary

#### LABSE

##### 40320 Samples (evaluate every 512 samples)

In [ ]:
labse_random = SentenceTransformer("sentence-transformers/LaBSE")

sample_size = 40320

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_fr_mixed_sentences,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)

train_loss = losses.MultipleNegativesRankingLoss(model=labse_random)


epochs = 1

custom_evaluator = CustomEvaluator(model_name = "LABSE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="labse_evaluation_logs_fr_mixed.jsonl")

# Evaluate before training starts (step 0)
custom_evaluator(model=labse_random, output_path="./model_output", epoch=0, steps=0)

labse_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=502,  # Evaluate every 502 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to labse_evaluation_logs_fr_mixed.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5022 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 502 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 1004 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 1506 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 2008 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 2510 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 3012 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 3514 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 4016 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 4518 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 5020 to labse_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step -1 to labse_evaluation_logs_fr_mixed.jsonl


In [ ]:
files.download("labse_evaluation_logs_fr_mixed.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### LuxEmbedder

##### 40320 Samples (evaluate every 512 samples)

In [ ]:
luxemb_random = SentenceTransformer("fredxlpy/LuxEmbedder")

sample_size = 40320

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_fr_mixed_sentences,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)

total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)


train_loss = losses.MultipleNegativesRankingLoss(model=luxemb_random)

custom_evaluator = CustomEvaluator(model_name = "LuxEmb", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en],  log_file="luxembed_evaluation_logs_fr_mixed.jsonl")
# Fine-tune the model
epochs = 1
# Evaluate before training starts (step 0)
custom_evaluator(model=luxemb_random, output_path="./model_output", epoch=0, steps=0)
# Fine-tune the model with evaluation every 500 steps
luxemb_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=504,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to luxembed_evaluation_logs_fr_mixed.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5022 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 504 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 1008 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 1512 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 2016 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 2520 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 3024 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 3528 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 4032 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 4536 to luxembed_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step -1 to luxembed_evaluation_logs_fr_mixed.jsonl


In [ ]:
files.download("luxembed_evaluation_logs_fr_mixed.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### M-GTE

##### 40320 Samples (evaluate every 512 samples)

In [ ]:
mgte_random = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

sample_size = 40320

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_fr_mixed_sentences,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.001 * total_steps)


custom_evaluator = CustomEvaluator(model_name = "M-GTE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="mgte_evaluation_logs_fr_mixed.jsonl")

train_loss = losses.MultipleNegativesRankingLoss(model=mgte_random)

# Fine-tune the model
epochs = 1

# Evaluate before training starts (step 0)
custom_evaluator(model=mgte_random, output_path="./model_output", epoch=0, steps=0)

# Fine-tune the model with evaluation every 500 steps
mgte_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=502,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Logged evaluation at epoch 0, step 0 to mgte_evaluation_logs_fr_mixed.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5022 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 502 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 1004 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 1506 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 2008 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 2510 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 3012 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 3514 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 4016 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 4518 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step 5020 to mgte_evaluation_logs_fr_mixed.jsonl
Logged evaluation at epoch 0, step -1 to mgte_evaluation_logs_fr_mixed.jsonl


In [ ]:
files.download("mgte_evaluation_logs_fr_mixed.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## English

### Prepare Training Set

In [ ]:
# Example usage to load the datasets
train_file_path = "lb_en_training_set.jsonl"

# Loading datasets
lb_en_training_set_loaded = load_dataset_local(train_file_path) # Or use the HF Version

# Confirm loading was successful
print(f"Loaded training set size (in articles): {len(lb_en_training_set_loaded)}")

cont_df = load_dataset("fredxlpy/LuxAlign", "lb-en", split="train")
lb_en_cont_training_sentences = [
    (lb, en) for lb, en in zip(cont_df["lb"], cont_df["en"])
    if lb and en
]

# Extract parallel sentences from the training lines
lb_en_training_sentences = extract_parallel_sentences(lb_en_training_set_loaded, src_col="lb", tgt_col="en")
# Prepare the training samples
lb_en_sampled_training_set = prepare_training_samples(lb_en_training_sentences, sample_size=19504, random_sampling=True)
# Prepare the training samples
lb_en_cont_training_set = prepare_training_samples(lb_en_cont_training_sentences, sample_size=20816, random_sampling=True)

lb_en_mixed_sentences = lb_en_sampled_training_set + lb_en_cont_training_set

Loaded training set size (in articles): 2105
Loaded test set size (in articles): 233


lb_en.json:   0%|          | 0.00/7.34M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28172 [00:00<?, ? examples/s]

### Historical Only

#### LABSE

##### 20160 Samples (evaluate every 512 samples)

In [ ]:
labse_random = SentenceTransformer("sentence-transformers/LaBSE")

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_en_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)

train_loss = losses.MultipleNegativesRankingLoss(model=labse_random)


epochs = 1

custom_evaluator = CustomEvaluator(model_name = "LABSE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="labse_evaluation_logs_en.jsonl")

# Evaluate before training starts (step 0)
custom_evaluator(model=labse_random, output_path="./model_output", epoch=0, steps=0)

labse_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to labse_evaluation_logs_en.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2438 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 64 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 128 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 192 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 256 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 320 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 384 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 448 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 512 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 576 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 640 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 704 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 768 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 832 to labse_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 896 to labse_eval

In [ ]:
files.download("labse_evaluation_logs_en.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### LuxEmbedder

##### 20160 Samples (evaluate every 512 samples)

In [ ]:
luxemb_random = SentenceTransformer("fredxlpy/LuxEmbedder")

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_en_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)

total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)


train_loss = losses.MultipleNegativesRankingLoss(model=luxemb_random)

custom_evaluator = CustomEvaluator(model_name = "LuxEmb", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en],  log_file="luxembed_evaluation_logs_en.jsonl")
# Fine-tune the model
epochs = 1
# Evaluate before training starts (step 0)
custom_evaluator(model=luxemb_random, output_path="./model_output", epoch=0, steps=0)
# Fine-tune the model with evaluation every 500 steps
luxemb_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to luxembed_evaluation_logs_en.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2438 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 64 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 128 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 192 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 256 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 320 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 384 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 448 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 512 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 576 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 640 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 704 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 768 to luxembed_evaluation_logs_en.jsonl
Logged evaluation at epoch 0, step 832 to luxembed_evaluation_logs_en.jsonl
Logged evalua

In [ ]:
files.download("luxembed_evaluation_logs_en.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### M-GTE

##### 20160 Samples (evaluate every 512 samples)

In [ ]:
mgte_random = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

sample_size = 20160

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_en_sampled_training_set,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.001 * total_steps)


custom_evaluator = CustomEvaluator(model_name = "M-GTE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="mgte_evaluation_logs_en.jsonl")

train_loss = losses.MultipleNegativesRankingLoss(model=mgte_random)

# Fine-tune the model
epochs = 1

# Evaluate before training starts (step 0)
custom_evaluator(model=mgte_random, output_path="./model_output", epoch=0, steps=0)

# Fine-tune the model with evaluation every 500 steps
mgte_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=64,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

In [ ]:
files.download("mgte_evaluation_logs_en.jsonl")

### Historical and Contemporary

#### LABSE

##### 40320 Samples (evaluate every 512 samples)

In [ ]:
labse_random = SentenceTransformer("sentence-transformers/LaBSE")

sample_size = 40320

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_en_mixed_sentences,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)

train_loss = losses.MultipleNegativesRankingLoss(model=labse_random)


epochs = 1

custom_evaluator = CustomEvaluator(model_name = "LABSE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="labse_evaluation_logs_en_mixed.jsonl")

# Evaluate before training starts (step 0)
custom_evaluator(model=labse_random, output_path="./model_output", epoch=0, steps=0)

labse_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=504,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to labse_evaluation_logs_en_mixed.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5040 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 504 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 1008 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 1512 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 2016 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 2520 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 3024 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 3528 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 4032 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 4536 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 5040 to labse_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step -1 to labse_evaluation_logs_en_mixed.jsonl


In [ ]:
files.download("labse_evaluation_logs_en_mixed.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### LuxEmbedder

##### 40320 Samples (evaluate every 512 samples)

In [ ]:
luxemb_random = SentenceTransformer("fredxlpy/LuxEmbedder")

sample_size = 40320

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_en_mixed_sentences,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)

total_steps = len(train_dataloader) * 1
warmup_steps = int(0.01 * total_steps)


train_loss = losses.MultipleNegativesRankingLoss(model=luxemb_random)

custom_evaluator = CustomEvaluator(model_name = "LuxEmb", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en],  log_file="luxembed_evaluation_logs_en_mixed.jsonl")
# Fine-tune the model
epochs = 1
# Evaluate before training starts (step 0)
custom_evaluator(model=luxemb_random, output_path="./model_output", epoch=0, steps=0)
# Fine-tune the model with evaluation every 500 steps
luxemb_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=504,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Logged evaluation at epoch 0, step 0 to luxembed_evaluation_logs_en_mixed.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5040 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 504 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 1008 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 1512 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 2016 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 2520 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 3024 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 3528 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 4032 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 4536 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 5040 to luxembed_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step -1 to luxembed_evaluation_logs_en_mixed.jsonl


In [ ]:
files.download("luxembed_evaluation_logs_en_mixed.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### M-GTE

##### 40320 Samples (evaluate every 512 samples)

In [ ]:
mgte_random = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

sample_size = 40320

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    lb_en_mixed_sentences,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.001 * total_steps)


custom_evaluator = CustomEvaluator(model_name = "M-GTE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="mgte_evaluation_logs_en_mixed.jsonl")

train_loss = losses.MultipleNegativesRankingLoss(model=mgte_random)

# Fine-tune the model
epochs = 1

# Evaluate before training starts (step 0)
custom_evaluator(model=mgte_random, output_path="./model_output", epoch=0, steps=0)

# Fine-tune the model with evaluation every 500 steps
mgte_random.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=504,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Logged evaluation at epoch 0, step 0 to mgte_evaluation_logs_en_mixed.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5040 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 504 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 1008 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 1512 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 2016 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 2520 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 3024 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 3528 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 4032 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 4536 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step 5040 to mgte_evaluation_logs_en_mixed.jsonl
Logged evaluation at epoch 0, step -1 to mgte_evaluation_logs_en_mixed.jsonl


In [ ]:
files.download("mgte_evaluation_logs_en_mixed.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## All-Together (Total 120K)

### Prepare Training Sets

In [ ]:
cont_df_fr = load_dataset("fredxlpy/LuxAlign", "lb-fr", split="train")

# Prepare the training samples
lb_fr_cont_training_set = prepare_training_samples(lb_fr_cont_training_sentences, sample_size=40320, random_sampling=True)

lb_fr_cont_training_sentences = [
    (lb, fr) for lb, fr in zip(cont_df_fr["lb"], cont_df_fr["fr"])
    if lb and fr
]
all_mixed_sentences =  lb_fr_sampled_training_set + lb_fr_cont_training_set + lb_de_sampled_training_set + lb_en_mixed_sentences

### M-GTE

In [ ]:
mgte_final = SentenceTransformer("Alibaba-NLP/gte-multilingual-base", trust_remote_code=True)

# Fine-tuning setup
batch_size = 8
# Create DataLoader
train_dataloader = DataLoader(
    all_mixed_sentences,
    shuffle=True,
    batch_size=batch_size,
    generator=torch.Generator().manual_seed(RANDOM_SEED),
)


total_steps = len(train_dataloader) * 1
warmup_steps = int(0.001 * total_steps)


custom_evaluator = CustomEvaluator(model_name = "M-GTE", sib_test_dataset=sib_test_dataset, class_to_templates=class_to_templates, similarity_data=paralux_data,  bitext_mining_datasets_de_lb=[bitext_mining_data_de_to_lb, bitext_mining_data_lb_to_de], bitext_mining_datasets_fr_lb=[bitext_mining_data_fr_to_lb, bitext_mining_data_lb_to_fr], bitext_mining_datasets_en_lb=[bitext_mining_data_en_to_lb, bitext_mining_data_lb_to_en], log_file="mgte_evaluation_logs_all_final.jsonl")

train_loss = losses.MultipleNegativesRankingLoss(model=mgte_final)

# Fine-tune the model
epochs = 1

# Evaluate before training starts (step 0)
custom_evaluator(model=mgte_final, output_path="./model_output", epoch=0, steps=0)

# Fine-tune the model with evaluation every 500 steps
mgte_final.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    evaluator=custom_evaluator,
    evaluation_steps=520,  # Evaluate every 64 steps (512 samples)
    show_progress_bar=True,
    output_path="./model_output",  # Save model and results to this path
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/123k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Logged evaluation at epoch 0, step 0 to mgte_evaluation_logs_all_final.jsonl


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15093 [00:00<?, ?it/s]

Logged evaluation at epoch 0, step 520 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 1040 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 1560 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 2080 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 2600 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 3120 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 3640 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 4160 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 4680 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 5200 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 5720 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 6240 to mgte_evaluation_logs_all_final.jsonl
Logged evaluation at epoch 0, step 6760 t

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive

# Define source and destination folder paths
source_folder = "/content/model_output"  # Change this to the folder you want to copy
destination_folder = "/content/drive/My Drive/mgte_120k"  # Change this to your Drive location

# Use the 'cp' command to copy the folder
!cp -r "{source_folder}" "{destination_folder}"

print("Folder copied successfully!")

Mounted at /content/drive
Folder copied successfully!
